In [3]:
import time
import re
import unicodedata
from dataclasses import dataclass
from fake_useragent import UserAgent
from slugify import slugify
from requests_html import HTML
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

In [4]:
# %pip install slugify

In [5]:
# def get_user_agent():
#     return UserAgent().random

# def extract_price_from_string(value: str, regex=r"[\$]{1}[\d,]+\.?\d{0,2}"):
#     x = re.findall(regex, value)
#     return x[0] if x else None

# def contains_url(text):
#     url_patterns = [r"http[s]?://", r"www\.", r"\.com", r"\.net", r"\.org"]
#     return any(re.search(pattern, text) for pattern in url_patterns)

# def extract_reviews(review_blocks):
#     all_reviews = []
#     for el in review_blocks:
#         span = el.find('span', first=True)
#         if span:
#             review_text = span.text.strip()
#             if not contains_url(review_text):
#                 all_reviews.append(review_text)
#     return all_reviews

# @dataclass
# class Scraper:
#     url: str = None
#     asin: str = None
#     endless_scroll: bool = False
#     endless_scroll_time: int = 5
#     driver = None
#     html_obj = None

#     def __post_init__(self):
#         if not self.url and self.asin:
#             self.url = f"https://www.amazon.com/dp/{self.asin}"
#         if not self.url:
#             raise ValueError("Provide either a valid URL or ASIN.")

#     def get_driver(self):
#         if self.driver is None:
#             options = uc.ChromeOptions()
#             options.add_argument("--headless")
#             options.add_argument("--no-sandbox")
#             options.add_argument("--disable-blink-features=AutomationControlled")
#             options.add_argument(f"user-agent={get_user_agent()}")
#             self.driver = uc.Chrome(options=options)
#         return self.driver

#     def perform_endless_scroll(self, driver):
#         if not self.endless_scroll:
#             return
#         last_height = driver.execute_script("return document.body.scrollHeight")
#         while True:
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             time.sleep(self.endless_scroll_time)
#             new_height = driver.execute_script("return document.body.scrollHeight")
#             if new_height == last_height:
#                 break
#             last_height = new_height

#     def get_html_obj(self):
#         if self.html_obj is None:
#             html_str = self.get()
#             self.html_obj = HTML(html=html_str)
#         return self.html_obj

#     def get(self):
#         driver = self.get_driver()
#         driver.get(self.url)
#         time.sleep(5)  # Allow time for CAPTCHA to potentially show up

#         if "captcha" in driver.current_url.lower():
#             print("⚠️ CAPTCHA detected! Consider using a proxy or rotating IPs.")
#             driver.quit()
#             raise Exception("CAPTCHA encountered.")

#         if self.endless_scroll:
#             self.perform_endless_scroll(driver)
#         else:
#             time.sleep(3)
#         return driver.page_source

#     def extract_element_text(self, css_selector):
#         html = self.get_html_obj()
#         el = html.find(css_selector, first=True)
#         return el.text.strip() if el else ""

#     def extract_tables(self):
#         html = self.get_html_obj()
#         return html.find("table")

#     def extract_table_dataset(self, tables) -> dict:
#         dataset = {}
#         for table in tables:
#             for tbody in table.element.getchildren():
#                 for tr in tbody.getchildren():
#                     row = [col.text_content().strip() for col in tr.getchildren() if col.text_content().strip()]
#                     if len(row) != 2:
#                         continue
#                     key, value = row
#                     print(key, value)
#                     key = slugify(key).replace("-", "_")
#                     if key in dataset:
#                         continue
#                     if "$" in value:
#                         dataset[key] = extract_price_from_string(value)
#                         dataset[f"{key}_raw"] = value
#                     else:
#                         dataset[key] = value
#         return dataset

#     def scrape(self):
#         html = self.get_html_obj()

#         title = self.extract_element_text("#productTitle")
#         price = (
#             self.extract_element_text("#priceblock_ourprice") or
#             self.extract_element_text("#priceblock_dealprice")
#         )

#         # Extract image URL
#         img_wrapper = html.find("#imgTagWrapperId img", first=True)
#         image_url = img_wrapper.attrs.get("src") if img_wrapper else None

#         # Extract rating and total reviews
#         rating = None
#         total_reviews = None
#         customer_reviews = html.find("#acrCustomerReviewText", first=True)
#         if customer_reviews:
#             review_text = customer_reviews.text.strip()
#             match = re.search(r'([\d,]+)', review_text)
#             if match:
#                 total_reviews = int(match.group(1).replace(',', ''))

#         rating_element = html.find("i.a-icon-star span.a-icon-alt", first=True)
#         if rating_element:
#             rating = rating_element.text.strip()

#         # Extract reviews
#         review_blocks = html.find('[data-hook="review-collapsed"]')
#         reviews = extract_reviews(review_blocks)

#         tables = self.extract_tables()
#         specs = self.extract_table_dataset(tables)
#         total = specs.get("total_raw")
#         # print(total_price)
#         print("Total Raw:", specs.get("total_raw"))
#         # for i, x in enumerate(dict(specs).items()):
#         #     print(i, x.get('total_raw'))
#         #     print(i, x)
#         return {
#             "title": title,
#             "price": price,
#             "total": total,
#             "image_url": image_url,
#             "rating": rating,
#             "total_reviews": total_reviews,
#             "reviews": reviews,
#             **specs
#         }


In [6]:
def get_user_agent():
    return UserAgent().random


def slugify(value: str) -> str:
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value).strip().lower()
    return re.sub(r'[-\s]+', '-', value)


def extract_price_from_string(value: str, regex=r"[\$]{1}[\d,]+\.?\d{0,2}"):
    x = re.findall(regex, value)
    return x[0] if x else None


def contains_url(text):
    url_patterns = [r"http[s]?://", r"www\.", r"\.com", r"\.net", r"\.org"]
    return any(re.search(pattern, text) for pattern in url_patterns)


def extract_reviews(review_blocks):
    all_reviews = []
    for el in review_blocks:
        span = el.find('span', first=True)
        if span:
            review_text = span.text.strip()
            if not contains_url(review_text):
                all_reviews.append(review_text)
    return all_reviews


@dataclass
class Scraper:
    url: str = None
    asin: str = None
    endless_scroll: bool = False
    endless_scroll_time: int = 5
    driver = None
    html_obj = None

    def __post_init__(self):
        if not self.url and self.asin:
            self.url = f"https://www.amazon.com/dp/{self.asin}"
        if not self.url:
            raise ValueError("Provide either a valid URL or ASIN.")

    def get_driver(self):
        if self.driver is None:
            options = uc.ChromeOptions()
            options.add_argument("--headless")
            options.add_argument("--no-sandbox")
            options.add_argument("--disable-blink-features=AutomationControlled")
            options.add_argument(f"user-agent={get_user_agent()}")
            self.driver = uc.Chrome(options=options)
        return self.driver

    def perform_endless_scroll(self, driver):
        if not self.endless_scroll:
            return
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(self.endless_scroll_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    def get_html_obj(self):
        if self.html_obj is None:
            html_str = self.get()
            self.html_obj = HTML(html=html_str)
        return self.html_obj

    def get(self):
        driver = self.get_driver()
        driver.get(self.url)
        time.sleep(5)  # Allow time for CAPTCHA to potentially show up

        if "captcha" in driver.current_url.lower():
            print("⚠️ CAPTCHA detected! Consider using a proxy or rotating IPs.")
            driver.quit()
            raise Exception("CAPTCHA encountered.")

        if self.endless_scroll:
            self.perform_endless_scroll(driver)
        else:
            time.sleep(3)
        return driver.page_source

    def extract_element_text(self, css_selector):
        html = self.get_html_obj()
        el = html.find(css_selector, first=True)
        return el.text.strip() if el else ""

    def extract_tables(self):
        html = self.get_html_obj()
        return html.find("table")

    def extract_table_dataset(self, tables) -> dict:
        dataset = {}
        for table in tables:
            for tbody in table.element.getchildren():
                for tr in tbody.getchildren():
                    row = [col.text_content().strip() for col in tr.getchildren() if col.text_content().strip()]
                    if len(row) != 2:
                        continue
                    key, value = row
                    print(key, value)
                    key = slugify(key).replace("-", "_")
                    if key in dataset:
                        continue
                    if "$" in value:
                        dataset[key] = extract_price_from_string(value)
                        dataset[f"{key}_raw"] = value
                    else:
                        dataset[key] = value
        return dataset

    def scrape(self):
        html = self.get_html_obj()

        title = self.extract_element_text("#productTitle")
        price = (
            self.extract_element_text("#priceblock_ourprice") or
            self.extract_element_text("#priceblock_dealprice")
        )

        # Extract image URL
        img_wrapper = html.find("#imgTagWrapperId img", first=True)
        image_url = img_wrapper.attrs.get("src") if img_wrapper else None

        # Extract rating and total reviews
        rating = None
        total_reviews = None
        customer_reviews = html.find("#acrCustomerReviewText", first=True)
        if customer_reviews:
            review_text = customer_reviews.text.strip()
            match = re.search(r'([\d,]+)', review_text)
            if match:
                total_reviews = int(match.group(1).replace(',', ''))

        rating_element = html.find("i.a-icon-star span.a-icon-alt", first=True)
        if rating_element:
            rating = rating_element.text.strip()

        # Extract reviews
        review_blocks = html.find('[data-hook="review-collapsed"]')
        reviews = extract_reviews(review_blocks)

        tables = self.extract_tables()
        specs = self.extract_table_dataset(tables)
        total = specs.get("total_raw")

        return {
            "title": title,
            "price": price,
            "total": total,
            "image_url": image_url,
            "rating": rating,
            "total_reviews": total_reviews,
            "reviews": reviews,
            **specs
        }





In [7]:
asin = "B0CLF3VPMV"

In [8]:
scraper = Scraper(asin=asin)
data = scraper.scrape()

Price $159.99
AmazonGlobal Shipping $11.74
Estimated Import Charges $9.04
Total $180.77
Price $159.99
AmazonGlobal Shipping $11.74
Estimated Import Charges $9.04
Total $180.77
Brand SAMSUNG
Model Name A9+
Memory Storage Capacity 64 GB
Screen Size 11 Inches
Display Resolution Maximum 1920 X 1200 (WUXGA) Pixels
Standing screen display size ‎11 Inches
Screen Resolution ‎1920 x 1200
Max Screen Resolution ‎1920 X 1200 (WUXGA) Pixels
RAM ‎4 GB
Hard Drive ‎64 GB
Chipset Brand ‎Qualcomm
Card Description ‎Integrated
Wireless Type ‎802.11ac
Brand ‎SAMSUNG
Series ‎A9+
Item model number ‎SM-X210NZAAXAR
Operating System ‎Android
Item Weight ‎1.32 pounds
Product Dimensions ‎10.12 x 6.64 x 0.27 inches
Item Dimensions  LxWxH ‎10.12 x 6.64 x 0.27 inches
Color ‎Graphite
Processor Brand ‎Qualcomm
Computer Memory Type ‎Unknown
Flash Memory Size ‎64 GB
Batteries ‎1 Lithium Ion batteries required.
ASIN B0CLF3VPMV
Customer Reviews 4.5  4.5 out of 5 stars    
    11,785 ratings   
                    
       

In [9]:
data

{'title': 'Samsung Galaxy Tab A9+ Tablet 11” 64GB Android Tablet, Big Screen, Quad Speakers, Upgraded Chipset, Multi Window Display, Slim, Light, Durable Design, US Version, 2024, Graphite',
 'price': '$159.99',
 'total': '$180.77',
 'image_url': 'https://m.media-amazon.com/images/I/61d46oYQgdL._AC_SX466_.jpg',
 'rating': '4.5 out of 5 stars',
 'total_reviews': 11785,
 'reviews': ['I had an older Galaxy Tab A (2019), but a friend broke it. Android tablet computers have really improved since I bought the old Samsung.\nI have a Samsung laptop and smartphone; they connect seamlessly and transferring data and media is a breeze.\nI knew the new tablet would be another Samsung.\nWhat I was unprepared for was how much I LOVE this Galaxy Tab A9+ (2024 version). The A9+ was released in 2023 and 2024. The latter has an improved chipset (Snapdragon) and is updated to Android 14. Otherwise, they are identical.\nThe A9+ has an 11" screen and is lightweight and thin enough to put a cover on and carr